<a href="https://colab.research.google.com/github/d-tomas/workshops/blob/main/20210217/notebooks/aprendizaje_automatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje automático

En este *notebook* vamos a desarrollar un sistema para análisis de sentimientos (clasificación) y otro para predecir el precio de una casa (regresión).

## Pasos previos

In [ ]:
# Librerías

import matplotlib.pyplot as plt  # Para hacer gráficas
import numpy as np  # Obtener valores únicos en un vector
import pandas as pd
from sklearn.metrics import accuracy_score  # Calcular la precisión del clasificador
from sklearn.model_selection import train_test_split  # Separar el dataset en entrenamiento y test
from sklearn.metrics import confusion_matrix  # Sacar la matriz de confusión
from sklearn.metrics import mean_absolute_error  # Mean Absolut Error (MAE) para regresión
from sklearn.svm import SVC  # Algoritmo Support Vector Machines
from sklearn.feature_extraction.text import TfidfVectorizer  # Matriz de términos por documento con TF-IDF
from xgboost import XGBRegressor  # Algoritmo de regresión
import seaborn as sns  # Visualización del mapa de calor

# Descargamos el corpus para entrenar y evaluar el sistema de clasificación
!wget https://raw.githubusercontent.com/d-tomas/workshops/main/20210217/datasets/cell_phones.csv
# Descargamos el corpus para entrenar y evaluar el sistema de regresión
!wget https://raw.githubusercontent.com/d-tomas/workshops/main/20210217/datasets/houses.csv

## Clasificación

Vamos a contruir un sistema de **análisis de sentimientos**. Vamos a entrenar el sistema con un corpus (conjunto de datos) que consta de 1000 opiniones sobre teléfonos móbiles en inglés, 500 positivas (etiquetadas como `POS`) y 500 negativas (etiquetadas como `NEG`).



In [ ]:
# Vemos que pinta tiene el corpus de entrenamiento

!head cell_phones.csv

In [ ]:
# Crear el clasificador para el análisis de sentimientos

data_classification = pd.read_csv('cell_phones.csv')  # Cargamos los datos del fichero
corpus = data_classification['content'].values  # Guardamos los mensajes
y = data_classification['opinion'].values  # Guardamos las opiniones

# Tenemos que transformar las palabras en números
# Cada palabra del mensaje se representa por su TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Separamos el corpus en entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Usamos SVM como algoritmo para la clasificaión
model = SVC(kernel = 'linear')
model.fit(X_train, y_train)  # Entrenamos el modelo

# Hacemos la predicción sobre todo el conjunto de test
predictions = model.predict(X_test)

# Calculamos la precisión del algoritmo
print('Precisión: {:.2%}\n'.format(accuracy_score(predictions, y_test)))
print('Matriz de confusión:')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True, linewidth=3)
plt.yticks(rotation=0)
plt.show()


In [ ]:
# Si queremos probar el modelo con una nueva entrada

new_input = ['I love this phone!!']
# Tenemos que transformar el texto a números, como se hizo al entrenar
new_input = vectorizer.transform(new_input)
model.predict(new_input)  # Predecimos la etiqueta para la nueva entrada (POS o NEG)

## Regresión

In [ ]:
data_regression = pd.read_csv('houses.csv')
data_regression

In [ ]:
data_regression.info()

In [ ]:
y = data_regression['SalePrice']  # Objetivo a predecir (el precio de las casas)
X = data_regression.drop(labels='SalePrice', axis=1)  # Todas las características de cada casa (menos su precio)

# Las variables categoriales (las que no contienen números) deben convertirse a números
# Usamos la técnica 'one-hot-encoding'
X = pd.get_dummies(X)

# Creamos los conjuntos de entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Creamos el modelo pasándole algunos parámetros para ajustarlo y mejorar el rendimiento
model = XGBRegressor(colsample_bytree=0.6, learning_rate=0.015, max_depth=4, min_child_weight=3, n_estimators=3000, subsample=0.75, random_state=1)
model.fit(X_train, y_train)  # Entrenamos el modelo

# Hacemos la predicción sobre el conjunto de test
predictions = model.predict(X_test)

# Calculamos la precisión del algoritmo (MAE)
# Cuanto más pequeño sea este valor mejor
print("MAE: {:,.0f}".format(mean_absolute_error(predictions, y_test)))